In [1]:
import os

In [2]:
# Go to the directory of the script (MLOPS_Project\)

os.chdir("/home/nicola/Projects/MLOPS_Project")

In [3]:
os.getcwd()

'/home/nicola/Projects/MLOPS_Project'

In [4]:
from dataclasses import dataclass 
from pathlib import Path

/home/nicola/anaconda3/envs/MlOps/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path 
    unzip_dir : Path

In [6]:
from mlopsProject.constants import *
from mlopsProject.utils.common import read_yaml, create_directory

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directory([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        
        config = self.config.data_ingestion 

        create_directory([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import urllib.request as request
import zipfile 

from mlopsProject.logging import logger 
from mlopsProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        """
        Downloads the data from the source url and saves it in the data directory

        Function returns None
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file
            )
        
            logger.info(f"Downloaded {filename} with following info: \n{headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists of size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        
        Extracts the zip file into the data directory
        
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  
    
    def delete_local(self):
        """
        Deletes the local data file

        Function returns None
        """
        os.remove(self.config.local_data_file)
        logger.info(f"Deleted {self.config.local_data_file}")

In [11]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    data_ingestion.delete_local()
except Exception as e:
    raise e

[2024-01-28 16:33:04,730: INFO: common: file: config/config.yaml loaded correctly]
[2024-01-28 16:33:04,732: INFO: common: file: params.yaml loaded correctly]
[2024-01-28 16:33:04,733: INFO: common: directory artifacts created]
[2024-01-28 16:33:04,734: INFO: common: directory artifacts/data_ingestion created]
[2024-01-28 16:33:04,735: INFO: 2096869070: artifacts/data_ingestion/data.zip already exists of size: approx 9565.393 KB]
[2024-01-28 16:33:04,891: INFO: 2096869070: Deleted artifacts/data_ingestion/data.zip]
